# TP : Conception d'Algorithmes Génétiques avec GeneticSharp

Dans ce TP, nous allons concevoir un filtre de détection de bords en utilisant la bibliothèque **GeneticSharp**. 

**Objectif :** Approcher automatiquement un **filtre de détection de bords** réputé (ici un filtre Sobel) à l’aide d’un algorithme génétique. 

Au fil du TP, nous allons :

1. Définir un **chromosome** (classe `EdgeChromosome`) permettant de stocker la description d’un noyau de convolution.
2. Mettre en place une **fonction d’évaluation** (classe `EdgeFitness`) comparant le résultat de notre filtre avec un filtre de référence (Sobel).
3. Paramétrer et exécuter un **algorithme génétique** (AG) avec [GeneticSharp](https://github.com/giacomelli/GeneticSharp).

> **Rappel :** Un algorithme génétique se base sur la métaphore de l’évolution naturelle. Les individus (ici, des filtres de convolution) sont évalués par une fonction de fitness (notre capacité à détecter des bords). À chaque génération, on applique :
> - une **sélection** (sélectionner les meilleurs individus) ;
> - un **croisement** (mélanger les individus pour explorer d’autres zones de l’espace de solutions) ;
> - une **mutation** (petites modifications aléatoires pour injecter de la diversité).



## Technologies et Bibliothèques

- **GeneticSharp**  
  Une bibliothèque d'algorithmes génétiques pour C# qui permet de configurer et d'exécuter des GA sur diverses plateformes .NET.  
  Elle facilite la définition de chromosomes, de fonctions de fitness et d'opérateurs génétiques (sélection, croisement, mutation).

- **Emgu CV & SkiaSharp**  
  Emgu CV (wrapper OpenCV pour .NET) est utilisée pour le traitement d'images (filtrage, convolution, conversion en niveaux de gris, etc.).  
  SkiaSharp permet de visualiser les images directement dans le notebook.

Ces technologies s'intègrent dans une démarche pédagogique visant à montrer comment l'évolution peut être utilisée pour optimiser des filtres de convolution dans le domaine du traitement d'image.


In [1]:
// Dépendances nécessaires
#r "nuget: GeneticSharp, 3.1.4"
#r "nuget: System.Drawing.Common, 9.0.0"

#r "nuget: Emgu.CV, 4.9.0.5494"
#r "nuget: Emgu.CV.Bitmap, 4.9.0.5494"
#r "nuget: Emgu.CV.runtime.mini.windows, 4.9.0.5494"

#r "nuget: SkiaSharp, 2.88.3"

// Chargement des utilitaires SkiaSharp depuis le fichier partagé
#load "../Config/SkiaUtils.cs"

In [2]:
// Imports
using System;
using System.IO;
using System.Drawing;
using Microsoft.DotNet.Interactive;

using System.Diagnostics;
using System.Globalization;

using GeneticSharp;

using Emgu.CV;
using Emgu.CV.CvEnum;
using Emgu.CV.Structure;

public static void SaveImage(Mat image, string fileName)
{
    image.Save(fileName);
    Console.WriteLine($"Image enregistrée : {fileName}");
}

Console.WriteLine("Dépendances chargées.");

Dépendances chargées.


## Représentation du Filtre : Le Chromosome et les Gènes

Pour représenter un filtre de convolution, nous utilisons un **chromosome** dont chaque **gène** contient une petite matrice (noyau).  
La matrice complète du filtre est obtenue en **additionnant** les matrices issues de chacun des gènes.

**Avantages de cette approche :**  
- **Granularité :** Chaque gène représente une contribution élémentaire, permettant une évolution progressive.  
- **Diversité :** La mutation et le croisement s'effectuent sur des sous-matrices, facilitant l'exploration de l'espace des solutions.

Dans la classe `EdgeChromosome`, on définit notamment :  
- La taille fixe du noyau (par exemple, 7×7).  
- La génération aléatoire de chaque gène avec des valeurs comprises dans un intervalle donné.  
- Une méthode pour sommer les matrices des gènes et obtenir la matrice finale qui sera utilisée pour filtrer l'image.


In [3]:
// Représentation du filtre par convolution sous forme de chromosome
public class EdgeChromosome : ChromosomeBase
{
    private const int KernelSize = 7; // Taille de la matrice du noyau

    public EdgeChromosome(int length) : base(length)
    {
        // Initialisation des gènes
        for (int i = 0; i < Length; i++)
        {
            ReplaceGene(i, GenerateGene(i));
        }
    }

    // Création d'un nouveau chromosome pour la population
    public override IChromosome CreateNew()
    {
        return new EdgeChromosome(Length);
    }

    // Génération d'un gène contenant une matrice avec des perturbations asymétriques
    public override Gene GenerateGene(int geneIndex)
    {
        var rnd = RandomizationProvider.Current;
        var matrix = new int[KernelSize, KernelSize];

        for (int i = 0; i < KernelSize; i++)
        {
            for (int j = 0; j < KernelSize; j++)
            {
                matrix[i, j] = rnd.GetInt(-20, 20);
               
            }
        }
        return new Gene(matrix);
    }

    public int[,] GetCompleteMatrix()
{
    var completeMatrix = new int[KernelSize, KernelSize];

    // Ajouter les matrices des gènes
    foreach (var gene in GetGenes())
    {
        var matrix = (int[,])gene.Value;
        for (int i = 0; i < KernelSize; i++)
        {
            for (int j = 0; j < KernelSize; j++)
            {
                completeMatrix[i, j] += matrix[i, j];
            }
        }
    }

    // Normalisation dynamique si des valeurs extrêmes apparaissent
    int maxAbsValue = completeMatrix.Cast<int>().Select(Math.Abs).Max();
    if (maxAbsValue > 10) // Seulement si les valeurs dépassent un seuil
    {
        for (int i = 0; i < KernelSize; i++)
        {
            for (int j = 0; j < KernelSize; j++)
            {
                completeMatrix[i, j] = (int)(10.0 * completeMatrix[i, j] / maxAbsValue);
            }
        }
    }

    return completeMatrix;
}

}
Console.WriteLine("Chromosome défini.");


Chromosome défini.


Testons la création d'un chromosome, ses gènes, et la matrice résultante correspondante

In [4]:
var testChromosome = new EdgeChromosome(5); // 5 gènes
Console.WriteLine("Gènes générés :");
foreach (var gene in testChromosome.GetGenes())
{
    var matrix = (int[,])gene.Value;
    for (int i = 0; i < matrix.GetLength(0); i++)
    {
        for (int j = 0; j < matrix.GetLength(1); j++)
        {
            Console.Write($"{matrix[i, j]} ");
        }
        Console.WriteLine();
    }
    Console.WriteLine();
}

var testMatrix = testChromosome.GetCompleteMatrix();

Console.WriteLine("Matrice complète générée :");
for (int i = 0; i < testMatrix.GetLength(0); i++)
{
    for (int j = 0; j < testMatrix.GetLength(1); j++)
    {
        Console.Write($"{testMatrix[i, j]} ");
    }
    Console.WriteLine();
}


Gènes générés :
2 2 14 -7 -10 -12 11 
3 16 -16 13 -4 8 13 
6 -20 7 -5 7 -9 9 
-13 19 -8 9 -13 -18 -19 
-10 17 -16 -3 17 -12 4 
-18 3 15 -4 -9 -10 -2 
-20 -18 -18 -16 13 4 8 

-14 1 12 15 -2 9 -14 
3 10 -19 1 -10 7 18 
4 -13 16 -7 5 -1 12 
-17 14 -16 -19 7 -18 3 
9 1 -13 -9 14 -12 -16 
-15 -20 -10 11 -18 0 -15 
-4 11 -17 -1 -16 0 10 

-10 -7 15 3 -3 17 18 
3 19 18 -18 7 0 14 
-8 -6 -14 14 16 19 -9 
8 -11 18 15 -3 19 -11 
13 0 -11 7 -9 14 -11 
-1 10 -12 -3 -7 -8 -9 
-11 -8 -15 17 -1 2 -5 

-16 19 17 19 14 5 -12 
-3 -19 6 13 2 0 16 
9 12 -5 -5 6 -7 -20 
-14 -3 -9 -18 8 -5 13 
-13 -3 15 -11 -19 5 9 
18 0 -14 -11 3 8 -10 
-10 3 1 8 -10 -2 1 

1 -15 -17 -12 -14 -16 -17 
1 -15 -1 2 -9 12 -8 
-12 -12 18 -9 -13 5 -14 
-6 -18 7 -11 -16 13 -6 
11 19 11 -12 -12 -7 3 
-15 -11 -1 7 -2 -9 -5 
-1 -14 0 13 6 -6 14 

Matrice complète générée :
-6 0 7 3 -2 0 -2 
1 2 -2 2 -2 5 10 
0 -7 4 -2 3 1 -4 
-7 0 -1 -4 -3 -1 -3 
1 6 -2 -5 -1 -2 -2 
-5 -3 -4 0 -6 -3 -7 
-8 -4 -9 3 -1 0 5 


## Fonction d'Évaluation (Fitness)

La **fonction d'évaluation** mesure la capacité d'un filtre (issu d'un chromosome) à détecter les bords de l'image.  
Pour ce faire, le processus est le suivant :

1. **Application du filtre généré**  
   La matrice complète du chromosome est utilisée pour réaliser une convolution sur l'image originale.

2. **Comparaison avec un filtre de référence (Sobel)**  
   On applique également le filtre Sobel sur l'image originale pour obtenir une image de référence.

3. **Calcul du score**  
   La similarité entre l'image filtrée par le chromosome et l'image de référence est mesurée (par exemple, via une corrélation normalisée).  
   Une pénalisation est éventuellement appliquée pour éviter des filtres uniformes (où la somme des coefficients est trop faible).

Ainsi, la fonction d'évaluation guide l'algorithme génétique en attribuant un score aux individus, favorisant ceux qui se rapprochent le plus de la détection de bords souhaitée.


In [5]:
using Emgu.CV;
using Emgu.CV.CvEnum;
using Emgu.CV.Structure;
using System.Drawing;
using System.Runtime.InteropServices;

public class EdgeFitness : IFitness
{
    private readonly Mat _originalImage;
    private readonly Mat _referenceImage;

    public EdgeFitness(Bitmap originalImage)
    {
    _originalImage = BitmapToMat(originalImage);

    // Convertir en niveaux de gris
    if (_originalImage.NumberOfChannels > 1)
    {
        CvInvoke.CvtColor(_originalImage, _originalImage, ColorConversion.Bgr2Gray);
    }

    // Appliquer le filtre Sobel pour référence
    _referenceImage = new Mat();
    CvInvoke.Sobel(_originalImage, _referenceImage, DepthType.Cv64F, 1, 0);

    if (_referenceImage.NumberOfChannels != 1)
        {
            CvInvoke.CvtColor(_referenceImage, _referenceImage, ColorConversion.Bgr2Gray);
        }

    }


    public async Task DisplayImagesAsync()
    {
        // Afficher l'image originale
        SaveImage(_originalImage, "original.png");
        await SkiaUtils.ShowImage("original.png", _originalImage.Width, _originalImage.Height);

        // Afficher l'image de référence (filtre Sobel)
        SaveImage(_referenceImage, "reference.png");
        await SkiaUtils.ShowImage("reference.png", _referenceImage.Width, _referenceImage.Height);
    }


    

    private Mat ApplyFilter(EdgeChromosome chromosome, bool display = false)
    {
        var filterMatrix = chromosome.GetCompleteMatrix();
        var kernel = ArrayToMat(ConvertToFloat(filterMatrix));

        if (display)
        {   
            Console.WriteLine("Noyau généré :");
            for (int i = 0; i < kernel.Rows; i++)
            {
                for (int j = 0; j < kernel.Cols; j++)
                {
                    Console.Write($"{kernel.GetData().GetValue(i, j)} ");
                }
                Console.WriteLine();
            }
        }
        

        var sourceImage = _originalImage.Clone();
        if (sourceImage.Depth != DepthType.Cv32F)
        {
            sourceImage.ConvertTo(sourceImage, DepthType.Cv32F);
        }

        var filteredImage = new Mat(sourceImage.Rows, sourceImage.Cols, DepthType.Cv32F, 1);
        CvInvoke.Filter2D(sourceImage, filteredImage, kernel, new Point(-1, -1));

        // Aligner la profondeur avec _referenceImage
        if (filteredImage.Depth != _referenceImage.Depth)
        {
            filteredImage.ConvertTo(filteredImage, _referenceImage.Depth);
        }

        return filteredImage;
    }


    
    public double Evaluate(IChromosome chromosome)
{
    var filteredImage = ApplyFilter((EdgeChromosome)chromosome, false);

    // Convertir en niveaux de gris si nécessaire
    if (filteredImage.NumberOfChannels > 1)
    {
        CvInvoke.CvtColor(filteredImage, filteredImage, ColorConversion.Bgr2Gray);
    }

    if (_referenceImage.NumberOfChannels > 1)
    {
        CvInvoke.CvtColor(_referenceImage, _referenceImage, ColorConversion.Bgr2Gray);
    }

    // Assurer la même profondeur
    if (filteredImage.Depth != DepthType.Cv8U)
    {
        filteredImage.ConvertTo(filteredImage, DepthType.Cv8U);
    }

    if (_referenceImage.Depth != DepthType.Cv8U)
    {
        _referenceImage.ConvertTo(_referenceImage, DepthType.Cv8U);
    }

    // Redimensionner si nécessaire
    if (filteredImage.Size != _referenceImage.Size)
    {
        CvInvoke.Resize(filteredImage, filteredImage, _referenceImage.Size);
    }

    // Créer une matrice pour stocker le résultat
    var result = new Mat();

    // Appliquer la méthode de corrélation normalisée
    CvInvoke.MatchTemplate(filteredImage, _referenceImage, result, TemplateMatchingType.CcorrNormed);

    // Extraire le score de corrélation maximum
    double minVal = 0, maxVal = 0;
    Point minLoc = new Point(), maxLoc = new Point();
    CvInvoke.MinMaxLoc(result, ref minVal, ref maxVal, ref minLoc, ref maxLoc);

    // Pénalisation des filtres uniformes
    var filterMatrix = ((EdgeChromosome)chromosome).GetCompleteMatrix();
    var filterSum = filterMatrix.Cast<int>().Sum();
    var filterPenalty = Math.Abs(filterSum) < 1e-3 ? 1 : Math.Log10(Math.Abs(filterSum) + 1);

    // Retourner le score ajusté
    return maxVal / filterPenalty;
}



    public async Task<DisplayedValue> DisplayChromosomeResult(EdgeChromosome chromosome, string fileNamePrefix, int generation, DisplayedValue placeholder = null)
{
    try
    {
        var filteredImage = ApplyFilter(chromosome, true);

        // Sauvegarder l'image
        string fileName = $"{fileNamePrefix}_generation_{generation}.png";
        SaveImage(filteredImage, fileName);
        Console.WriteLine($"Image sauvegardée : {fileName}");

        if (placeholder != null)
        {
            placeholder.Update(await SkiaUtils.ShowImage(fileName, filteredImage.Width, filteredImage.Height));
            return placeholder;
        }

        return await SkiaUtils.ShowImage(fileName, filteredImage.Width, filteredImage.Height);
    }
    catch (Exception ex)
    {
        Console.WriteLine($"Erreur dans DisplayChromosomeResult : {ex.Message}");
        throw;
    }
}


    private Mat BitmapToMat(Bitmap bitmap)
    {
        // Crée un Mat vide avec les mêmes dimensions et type
        var mat = new Mat(bitmap.Height, bitmap.Width, DepthType.Cv8U, 3);

        // Bloquer les bits du Bitmap pour accéder directement aux données
        var bitmapData = bitmap.LockBits(
            new Rectangle(0, 0, bitmap.Width, bitmap.Height),
            System.Drawing.Imaging.ImageLockMode.ReadOnly,
            System.Drawing.Imaging.PixelFormat.Format24bppRgb);

        // Copier les données du Bitmap vers le Mat
        using (var image = new Image<Bgr, byte>(bitmap.Width, bitmap.Height, bitmapData.Stride, bitmapData.Scan0))
        {
            mat = image.Mat.Clone();
        }

        // Libérer les bits verrouillés
        bitmap.UnlockBits(bitmapData);

        return mat;
    }
    

   private Mat ArrayToMat(float[,] array)
    {
        var rows = array.GetLength(0);
        var cols = array.GetLength(1);
        var mat = new Mat(rows, cols, DepthType.Cv32F, 1);

        var data = new float[rows * cols];
        Buffer.BlockCopy(array, 0, data, 0, rows * cols * sizeof(float));
        mat.SetTo(data);

        return mat;
    }


    private float[,] ConvertToFloat(int[,] intArray)
    {
        var rows = intArray.GetLength(0);
        var cols = intArray.GetLength(1);
        var floatArray = new float[rows, cols];

        for (int i = 0; i < rows; i++)
        {
            for (int j = 0; j < cols; j++)
            {
                floatArray[i, j] = intArray[i, j];
            }
        }
        return floatArray;
    }
}


Error: (141,129): error CS0246: Le nom de type ou d'espace de noms 'DisplayedValue' est introuvable (vous manque-t-il une directive using ou une référence d'assembly ?)
(141,23): error CS0246: Le nom de type ou d'espace de noms 'DisplayedValue' est introuvable (vous manque-t-il une directive using ou une référence d'assembly ?)
(38,15): error CS0103: Le nom 'SkiaUtils' n'existe pas dans le contexte actuel
(42,15): error CS0103: Le nom 'SkiaUtils' n'existe pas dans le contexte actuel
(154,38): error CS0103: Le nom 'SkiaUtils' n'existe pas dans le contexte actuel
(158,22): error CS0103: Le nom 'SkiaUtils' n'existe pas dans le contexte actuel

Test de la fonction fitness

In [6]:
var imagePath = @"MRI_Prostate_Cancer.jpg"; 
var originalImage = (Bitmap)Image.FromFile(imagePath);

var fitness = new EdgeFitness(originalImage);

// Test d'un chromosome
var chromosome = new EdgeChromosome(20);
Console.WriteLine($"Score de fitness : {fitness.Evaluate(chromosome)}");


Error: (4,19): error CS0246: Le nom de type ou d'espace de noms 'EdgeFitness' est introuvable (vous manque-t-il une directive using ou une référence d'assembly ?)

## Configuration de l'Algorithme Génétique

L'algorithme génétique est configuré à l'aide de plusieurs éléments clés :

- **Population**  
  Définie par un nombre minimum et maximum d'individus.  
  Chaque individu est un chromosome représentant un filtre de convolution.

- **Opérateurs Génétiques**  
  - **Sélection :** Par exemple, l'`EliteSelection` qui conserve les meilleurs individus.  
  - **Croisement :** Par exemple, le `UniformCrossover` permettant de mélanger les gènes entre chromosomes.  
  - **Mutation :** Par exemple, la `ReverseSequenceMutation` qui inverse des séquences de gènes pour injecter de la diversité.

- **Critère d'Arrêt**  
  L'algorithme s'arrête après un nombre fixé de générations (par exemple, 100 générations).

Ces paramètres sont ajustables et permettent d'explorer l'influence de la diversité et de la sélection sur la qualité des solutions.


In [7]:
// Charger une image de test
var imagePath = @"MRI_Prostate_Cancer.jpg"; 
var originalImage = (Bitmap)Bitmap.FromFile(imagePath);

// await SkiaUtils.ShowImage(imagePath, originalImage.Width, originalImage.Height);

// Initialiser la fonction de fitness
var fitness = new EdgeFitness(originalImage);
await fitness.DisplayImagesAsync();

// Initialiser un chromosome
var chromosome = new EdgeChromosome(20); // Taille des chromosomes

// Initialiser la population
var population = new Population(50, 100, chromosome);

// Configurer les opérateurs génétiques
var selection = new EliteSelection();
var crossover = new UniformCrossover();
var mutation = new ReverseSequenceMutation();

// Configurer l'algorithme génétique
var ga = new GeneticAlgorithm(population, fitness, selection, crossover, mutation)
{
    Termination = new GenerationNumberTermination(100)


};


Console.WriteLine("Algorithme génétique configuré.");


Error: (8,19): error CS0246: Le nom de type ou d'espace de noms 'EdgeFitness' est introuvable (vous manque-t-il une directive using ou une référence d'assembly ?)

## Exécution et Visualisation

Pendant l'exécution de l'algorithme génétique, plusieurs aspects sont mis en avant :

- **Mise à jour itérative**  
  À chaque génération, le meilleur chromosome est évalué, et son filtre est appliqué à l'image originale.

- **Visualisation dynamique**  
  Grâce à SkiaSharp, le notebook affiche périodiquement l'image obtenue par le meilleur filtre, la comparaison avec le filtre de référence, ou encore la différence entre les deux.  
  Cette alternance visuelle permet de suivre l'évolution de la performance du GA.

- **Suivi des scores**  
  Les logs affichent le numéro de génération et le score de fitness du meilleur individu, offrant ainsi un aperçu quantitatif de la convergence.


In [8]:
// Initialiser le placeholder avec l'image originale
DisplayedValue imagePlaceholder = await SkiaUtils.ShowImage(imagePath, originalImage.Width, originalImage.Height);

ga.GenerationRan += (sender, e) =>
{
    var bestChromosome = ga.BestChromosome as EdgeChromosome;
    if (bestChromosome != null)
    {
        Console.WriteLine($"Génération {ga.GenerationsNumber} - Meilleur score : {bestChromosome.Fitness}");

        if (ga.GenerationsNumber % 10 == 0)
        {
            // Mettre à jour l'image via DisplayChromosomeResult
            Task.Run(async () =>
            {
                try
                {
                    imagePlaceholder = await fitness.DisplayChromosomeResult(bestChromosome, "best", ga.GenerationsNumber).ConfigureAwait(false);
                }
                catch (Exception ex)
                {
                    Console.WriteLine($"Erreur lors de la mise à jour de l'image : {ex.Message}");
                }
            });
        }
    }
};



Console.WriteLine("Lancement de l'algorithme génétique...");
ga.Start();
Console.WriteLine($"Meilleure solution trouvée avec un score de {ga.BestChromosome.Fitness}.");


Error: (2,1): error CS0246: Le nom de type ou d'espace de noms 'DisplayedValue' est introuvable (vous manque-t-il une directive using ou une référence d'assembly ?)
(2,41): error CS0103: Le nom 'SkiaUtils' n'existe pas dans le contexte actuel
(2,61): error CS0103: Le nom 'imagePath' n'existe pas dans le contexte actuel
(2,72): error CS0103: Le nom 'originalImage' n'existe pas dans le contexte actuel
(2,93): error CS0103: Le nom 'originalImage' n'existe pas dans le contexte actuel
(4,1): error CS0103: Le nom 'ga' n'existe pas dans le contexte actuel
(32,1): error CS0103: Le nom 'ga' n'existe pas dans le contexte actuel
(33,66): error CS0103: Le nom 'ga' n'existe pas dans le contexte actuel

## Conclusion et Perspectives

Ce TP a permis d'illustrer comment un algorithme génétique peut être appliqué pour optimiser un filtre de détection de bords en traitement d'image.  
L'approche par décomposition en gènes permet une évolution fine et progressive du filtre.

**Perspectives possibles :**  
- Modifier les paramètres du GA (taille de la population, taux de mutation, etc.) pour observer leur impact sur la convergence.  
- Expérimenter avec d'autres opérateurs génétiques ou représentations du problème.  
- Implémenter une version équivalente en Python avec PyGad pour comparer les approches.

Cette démarche pédagogique démontre la puissance des algorithmes évolutionnaires pour résoudre des problèmes complexes dans le domaine de la vision par ordinateur.
